<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/agent_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [3]:
!pip install --quiet -U langchain langchain_openai langgraph langchainhub langchain_experimental
!pip install google
!pip install wikipedia
!pip install duckduckgo-search
!pip install arxiv
!pip install python-dotenv
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.2/150.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=f0a51f99f9b52e73b4073566b918f967c69e996ad2bbcb30b72e1334ee1be57f
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011

# Import Libraries

In [4]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.messages import BaseMessage
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.file_management import ReadFileTool
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_core.output_parsers import StrOutputParser
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.llms import OpenAI
import random
import operator
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import getpass
from dotenv import load_dotenv
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"

Mounted at /content/drive


# Set up

In [5]:
!rm -rf .env
!touch .env
!cat /content/drive/MyDrive/openai_api_key.txt >> .env
!cat /content/drive/MyDrive/langchain_api_key.txt >> .env
!cat /content/drive/MyDrive/tavily_api_key.txt >> .env

In [6]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [7]:
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_01"
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key

# Experiments

## Simple agent

In [8]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}")
])
llm = OpenAI(temperature=0)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input": "what kind of product is CMCBL730P1 - CRAFTSMAN V20 BRUSHLESS BLOWER KIT? Answer just product type in a short concise way without any fluffs."})

<ipython-input-8-38689d6ebac9>:5: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


'\n\nProduct type: Cordless blower kit.'

## Websearch and RAG agent

In [9]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
docs[0]

Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'}, page_content="\n\n\n\n\nLangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingThis is outdated documentation for \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangS

In [10]:
# Loading in a vector database called FAISS
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})


'\n\nAnswer: Langsmith can help visualize test results.'

In [11]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])



Answer: LangSmith allows developers to create datasets and run tests on their LLM applications. It also provides a comparison view for tracking and diagnosing regressions in test scores, a playground environment for rapid iteration and experimentation, and the ability to collect feedback from users. Additionally, LangSmith supports sending runs to annotation queues for closer inspection and annotation, and allows for automations to be set up for processing traces in near real-time.


## Conversation Chain

In [12]:
# First we need a prompt that we can pass into an LLM to generate this search query
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [13]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(id='e6ac56f5-9754-4353-900d-6f8512527c4c', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'}, page_content='LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingThis is outdated documentation 

In [14]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [15]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})],
 'input': 'Tell me how',
 'context': [Document(id='e6ac56f5-9754-4353-900d-6f8512527c4c', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'}, page_content='LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conf

## Web Search

In [ ]:
search = TavilySearchResults()
tools = [retriever_tool, search]

In [47]:
# Get the prompt to use - you can modify this!
prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],
    template="You are a helpful assistant. {input} {agent_scratchpad}",
) #hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm,
                                               tools,
                                               prompt)

In [62]:
class AgentState(TypedDict):
   # The input string
   input: str
   # The list of previous messages in the conversation
   chat_history: list[BaseMessage]
   # The outcome of a given call to the agent
   # Needs `None` as a valid type, since this is what this will start as
   agent_outcome: Union[AgentAction, AgentFinish, None]
   # List of actions and corresponding observations
   # Here we annotate this with `operator.add` to indicate that operations to
   # this state should be ADDED to the existing values (not overwrite it)
   intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])


In [17]:
@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100."""
    return random.randint(0, 100)

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100."""
    return random.randint(0, 100)

@tool
def websearch_tool(query:str):
    """Use this tool to perform web searches using DuckDuckGo."""
    return DuckDuckGoSearchRun().run(query)

tools = [to_lower_case, random_number_maker, websearch_tool]

In [32]:
random_number_maker.run('random'), to_lower_case.run('SRILA'), websearch_tool.run('Tariff')

(15,
 'srila',
 'Learn what a tariff is, how it works, and why governments impose it. Explore the different types of tariffs, their advantages and disadvantages, and their impact on trade and the economy. However, a 10% tariff on all goods coming to the US from China took effect at 12:01 a.m. ET on Tuesday. China immediately retaliated with tariffs on some energy and metals imports. A country\'s tariff rates depend on its trade relationship with the US. The US is one of 164 member nations of the World Trade Organization (WTO), which has a framework for trade among its members. The WTO\'s "most favored nation" rule maintains that an import tariff set by a member country on another member must apply to all members at ... President Donald Trump in his inaugural speech credited President William McKinley for making the United States "very rich" through tariffs. Imagine a local market selling apples. The retailer can choose between locally grown apples and imported ones. If a foreign produc

In [52]:
inputs = {"input": "give me a random number and then write in words and make it lower case.",
          "chat_history": [],
          "intermediate_steps":[]}

agent_outcome = agent_runnable.invoke(inputs)

In [53]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='You are a helpful assistant. {input} {agent_scratchpad}')

In [54]:
prompt.get_prompts()

[PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, template='You are a helpful assistant. {input} {agent_scratchpad}')]

In [55]:
agent_outcome

AgentActionMessageLog(tool='random_number', tool_input={'input': 'random number'}, log="\nInvoking: `random_number` with `{'input': 'random number'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random number"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_2f141ce944'}, id='run-a2ac7222-cdfd-43e9-b8a2-2524562aee45-0')])

In [56]:
type(agent_runnable)

langchain_core.runnables.base.RunnableSequence

In [50]:
inputs = {"input": "What type of product is CMCBL730P1?",
          "chat_history": [],
          "intermediate_steps":[]}

agent_outcome = agent_runnable.invoke(inputs)

In [51]:
agent_outcome

AgentActionMessageLog(tool='websearch_tool', tool_input={'query': 'CMCBL730P1 product type'}, log="\nInvoking: `websearch_tool` with `{'query': 'CMCBL730P1 product type'}`\nresponded: Let me look that up for you.\n\n", message_log=[AIMessage(content='Let me look that up for you.', additional_kwargs={'function_call': {'arguments': '{"query":"CMCBL730P1 product type"}', 'name': 'websearch_tool'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_2f141ce944'}, id='run-6a089a49-60b4-4569-a47e-5a058d20f949-0')])